<a href="https://colab.research.google.com/github/Daleth-Barreto/Practica2_ML/blob/main/Practica2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rendimiento Estudientes

In [96]:
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import os

In [97]:
import pandas as pd

In [98]:
def cargar_csv_kagglehub(enlace_kagglehub):
    path = kagglehub.dataset_download(enlace_kagglehub)
    print("✅ Dataset descargado en:", path)

    # Buscar primer CSV dentro de la carpeta descargada
    for root, dirs, files_in_dir in os.walk(path):
        for file in files_in_dir:
            if file.endswith(".csv"):
                full_path = os.path.join(root, file)
                print("📄 Cargando:", file)
                return pd.read_csv(full_path)

    print("❌ No se encontró ningún archivo CSV en el dataset.")
    return None


In [121]:
students_df = cargar_csv_kagglehub("bhavikjikadara/student-study-performance")
students_df

✅ Dataset descargado en: /kaggle/input/student-study-performance
📄 Cargando: study_performance.csv


,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


##Preprocesado

###Variable a predecir: Score

In [122]:
score_colunmns = [columna for columna in students_df.columns if columna.endswith("score")]
score_colunmns

['math_score', 'reading_score', 'writing_score']

In [123]:
students_df['score'] = round(students_df[score_colunmns].sum(axis = 1)/3) # obtenemos el promedio y redondeamos
students_df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,score
0,female,group B,bachelor's degree,standard,none,72,72,74,73.0
1,female,group C,some college,standard,completed,69,90,88,82.0
2,female,group B,master's degree,standard,none,90,95,93,93.0
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.0
4,male,group C,some college,standard,none,76,78,75,76.0
...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,94.0
996,male,group C,high school,free/reduced,none,62,55,55,57.0
997,female,group C,high school,free/reduced,completed,59,71,65,65.0
998,female,group D,some college,standard,completed,68,78,77,74.0


In [124]:
#students_df.drop(columns=score_colunmns, inplace=True)
students_df = students_df.drop(columns=score_colunmns)
students_df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,score
0,female,group B,bachelor's degree,standard,none,73.0
1,female,group C,some college,standard,completed,82.0
2,female,group B,master's degree,standard,none,93.0
3,male,group A,associate's degree,free/reduced,none,49.0
4,male,group C,some college,standard,none,76.0
...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,94.0
996,male,group C,high school,free/reduced,none,57.0
997,female,group C,high school,free/reduced,completed,65.0
998,female,group D,some college,standard,completed,74.0


##Variables categóricas


###One hot encoding

In [125]:
from sklearn.preprocessing import OneHotEncoder

In [126]:
def one_hot_encoder(df, column_name):
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(df[[column_name]])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([column_name]))
    df = pd.concat([df, encoded_df], axis=1)
    df.drop(column_name, axis=1, inplace=True)
    return df

In [127]:
encoder = OneHotEncoder(sparse_output=False)


In [128]:
encoded_Data = encoder.fit_transform(students_df[['gender']])
encoded_Data

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [129]:
encoded_df = pd.DataFrame(encoded_Data, columns=encoder.get_feature_names_out(['gender']))
encoded_df

,gender_female,gender_male
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
995,1.0,0.0
996,0.0,1.0
997,1.0,0.0
998,1.0,0.0


In [130]:
students_df = pd.concat([students_df, encoded_df], axis=1)

students_df.drop('gender', axis=1, inplace=True)
students_df

,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,score,gender_female,gender_male
0,group B,bachelor's degree,standard,none,73.0,1.0,0.0
1,group C,some college,standard,completed,82.0,1.0,0.0
2,group B,master's degree,standard,none,93.0,1.0,0.0
3,group A,associate's degree,free/reduced,none,49.0,0.0,1.0
4,group C,some college,standard,none,76.0,0.0,1.0
...,...,...,...,...,...,...,...
995,group E,master's degree,standard,completed,94.0,1.0,0.0
996,group C,high school,free/reduced,none,57.0,0.0,1.0
997,group C,high school,free/reduced,completed,65.0,1.0,0.0
998,group D,some college,standard,completed,74.0,1.0,0.0


In [131]:
students_df = one_hot_encoder(students_df, 'test_preparation_course')
students_df

,race_ethnicity,parental_level_of_education,lunch,score,gender_female,gender_male,test_preparation_course_completed,test_preparation_course_none
0,group B,bachelor's degree,standard,73.0,1.0,0.0,0.0,1.0
1,group C,some college,standard,82.0,1.0,0.0,1.0,0.0
2,group B,master's degree,standard,93.0,1.0,0.0,0.0,1.0
3,group A,associate's degree,free/reduced,49.0,0.0,1.0,0.0,1.0
4,group C,some college,standard,76.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
995,group E,master's degree,standard,94.0,1.0,0.0,1.0,0.0
996,group C,high school,free/reduced,57.0,0.0,1.0,0.0,1.0
997,group C,high school,free/reduced,65.0,1.0,0.0,1.0,0.0
998,group D,some college,standard,74.0,1.0,0.0,1.0,0.0


In [132]:
students_df =one_hot_encoder(students_df, 'race_ethnicity')
students_df = one_hot_encoder(students_df, 'lunch')
students_df

,parental_level_of_education,score,gender_female,gender_male,test_preparation_course_completed,test_preparation_course_none,race_ethnicity_group A,race_ethnicity_group B,race_ethnicity_group C,race_ethnicity_group D,race_ethnicity_group E,lunch_free/reduced,lunch_standard
0,bachelor's degree,73.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,some college,82.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,master's degree,93.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,associate's degree,49.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,some college,76.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,master's degree,94.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
996,high school,57.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
997,high school,65.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
998,some college,74.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


###Ordinal encoding

In [133]:
students_df['parental_level_of_education'].value_counts()

,count
parental_level_of_education,
some college,226
associate's degree,222
high school,196
some high school,179
bachelor's degree,118
master's degree,59


In [134]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_categories = ["some high school", "high school", "some college", "associate's degree", "bachelor's degree", "master's degree"]
encoder = OrdinalEncoder(categories=[ordinal_categories])

encoded_data = encoder.fit_transform(students_df[['parental_level_of_education']])
df_encoded = pd.DataFrame(encoded_data, columns=['parental_level_of_education'])
students_df = pd.concat([students_df,df_encoded], axis=1)
students_df.drop('parental_level_of_education', axis=1, inplace=True)
students_df


,score,gender_female,gender_male,test_preparation_course_completed,test_preparation_course_none,race_ethnicity_group A,race_ethnicity_group B,race_ethnicity_group C,race_ethnicity_group D,race_ethnicity_group E,lunch_free/reduced,lunch_standard
0,73.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,82.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,93.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,49.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,76.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,94.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
996,57.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
997,65.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
998,74.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


##Machine Learning

In [138]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [143]:
X = students_df.drop('score', axis=1)
y = students_df['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)


In [157]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [158]:
rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)

In [159]:
rf_mse = mean_squared_error(y_test, rf_predictions)
lr_mse = mean_squared_error(y_test, lr_predictions)

print(f"Random Forest MSE: {rf_mse}")
print(f"Linear Regression MSE: {lr_mse}")

Random Forest MSE: 169.72346682508774
Linear Regression MSE: 166.23131268108227


In [160]:
print("\nSample Predictions:")

num_samples = 5
for i in range(num_samples):
  print(f" - Random Forest: Predicted Score = {rf_predictions[i]}, Actual Score = {y_test.iloc[i]}")
  print(f" - Linear Regression: Predicted Score = {lr_predictions[i]}, Actual Score = {y_test.iloc[i]}")


Sample Predictions:
 - Random Forest: Predicted Score = 69.06201124245538, Actual Score = 85.0
 - Linear Regression: Predicted Score = 68.80082502481247, Actual Score = 85.0
 - Random Forest: Predicted Score = 60.669949028611555, Actual Score = 44.0
 - Linear Regression: Predicted Score = 65.06631855899334, Actual Score = 44.0
 - Random Forest: Predicted Score = 51.01051262202424, Actual Score = 69.0
 - Linear Regression: Predicted Score = 55.20421119944869, Actual Score = 69.0
 - Random Forest: Predicted Score = 60.51613982756573, Actual Score = 51.0
 - Linear Regression: Predicted Score = 63.746571479569184, Actual Score = 51.0
 - Random Forest: Predicted Score = 61.32285441092875, Actual Score = 56.0
 - Linear Regression: Predicted Score = 59.05639379822943, Actual Score = 56.0


In [161]:
rf_predictions = rf_model.predict(X_test).round()
lr_predictions = lr_model.predict(X_test).round()

rf_mse = mean_squared_error(y_test, rf_predictions)
lr_mse = mean_squared_error(y_test, lr_predictions)

In [162]:
rf_mse = mean_squared_error(y_test, rf_predictions)
lr_mse = mean_squared_error(y_test, lr_predictions)

print(f"Random Forest MSE: {rf_mse}")
print(f"Linear Regression MSE: {lr_mse}")

print("\nSample Predictions:")

num_samples = 5
for i in range(num_samples):
  print(f" - Random Forest: Predicted Score = {rf_predictions[i]}, Actual Score = {y_test.iloc[i]}")
  print(f" - Linear Regression: Predicted Score = {lr_predictions[i]}, Actual Score = {y_test.iloc[i]}")

Random Forest MSE: 170.095
Linear Regression MSE: 165.845

Sample Predictions:
 - Random Forest: Predicted Score = 69.0, Actual Score = 85.0
 - Linear Regression: Predicted Score = 69.0, Actual Score = 85.0
 - Random Forest: Predicted Score = 61.0, Actual Score = 44.0
 - Linear Regression: Predicted Score = 65.0, Actual Score = 44.0
 - Random Forest: Predicted Score = 51.0, Actual Score = 69.0
 - Linear Regression: Predicted Score = 55.0, Actual Score = 69.0
 - Random Forest: Predicted Score = 61.0, Actual Score = 51.0
 - Linear Regression: Predicted Score = 64.0, Actual Score = 51.0
 - Random Forest: Predicted Score = 61.0, Actual Score = 56.0
 - Linear Regression: Predicted Score = 59.0, Actual Score = 56.0
